# Computable blog posts by [Quansight](https://www.quansight.com/)

At [Quansight](https://www.quansight.com/), we frequently use [Jupyter notebooks](jupyter.org) to communicate our on going contributions to [sustainable open source software](https://www.quansight.com/sos-partnership).  We thought it would be appropriate to compose blog posts with notebooks so they may be read as blog posts with [Jekyll](https://jekyllrb.com) and reused as demos on [Binder](https://mybinder.org/).

## Convert and deploy notebooks

In this notebook, we will build the tools required to convert a notebook to a valid [Jekyll post](https://jekyllrb.com/docs/posts/).  This process requires:
    
1. Converting notebooks to markdown with [`nbconvert`](http://nbconvert.readthedocs.io/).
2. Prepending [yaml front matter](https://jekyllrb.com/docs/front-matter/) to the markdown.
3. Setting up the [Travis Pages Deployment](https://docs.travis-ci.com/user/deployment/pages/).

`collect_last_input` is a utility to function to extract pieces of the last code cell that way run.

In [49]:
    def collect_last_input(stop=-1):
        try: return '\n'.join(In[-1].splitlines()[:stop])
        except: return 

### Dependencies

* [`nbconvert`](http://nbconvert.readthedocs.io/) will convert notebooks into other file formats.
* [`git`/__GitPython__](https://gitpython.readthedocs.io/en/stable/) allows us to introspect the revision history to discover metadata about a post.
* [`yaml`/__PyYAML__](https://pyyaml.org/) will compose the [YAML Front Matter](https://jekyllrb.com/docs/front-matter/) that Jekyll uses to create posts.

In [50]:
    import nbconvert, git, yaml, inspect; from pathlib import Path
    _imports = collect_last_input()

[__jupyter_nbconvert_config.py__](https://nbconvert.readthedocs.io/en/latest/config_options.html) is a special file name recognized by the `nbconvert` `configuration` system.  A goal of this notebook build the correct configuration file to create our posts.

In [51]:
    configuration = Path('..', 'jupyter_nbconvert_config.py')

`FrontMatters` converts a notebook to Markdown and preprends the `yaml` front matter.

In [67]:
    class FrontMatters(nbconvert.exporters.MarkdownExporter):
        def from_notebook_node(self, nb, resources=None, **kw):
            nb, resources = super().from_notebook_node(nb, resources, **kw)
            md = dict(resources['metadata'])
            md['author'] = author_from_repo(Path(md['path'], f"{md['name']}.ipynb"))
            md['layout'] = 'post'
            return '---\n'.join((
                '', yaml.safe_dump(md, default_flow_style=False), nb)), resources
    _front_matter_source = collect_last_input()

`author_from_repo` extracts the authors of a notebook from the `git` revision history.

In [68]:
    def author_from_repo(file, dir='.'):
        repo = git.Repo(dir)
        return repo.blame('HEAD~0', file)[0][0].author.name

## Formatting __jupyter_nbconvert_config.py__

The components above are combined to create our `configuration` file.

In [69]:
    collect_last_input() and configuration.write_text (F"""{_imports}

    {_front_matter_source}

    {inspect.getsource(author_from_repo)}
    try:
        c.NbConvertApp.export_format = f"jupyter_nbconvert_config.FrontMatters"
        c.FilesWriter.build_directory = "_posts"
    except: ...""");

## Tests

Below we test that our `configuration` exports the correct features to 

In [ ]:
    def test_author():
        assert author_from_repo('_notebooks/2018-09-14-Computable-posts-from-quansight.ipynb', dir='..') == 'Tony Fast'
        
    def test_convert():
        from IPython import get_ipython
        import io
        !source activate p6 && pushd .. && jupyter nbconvert _notebooks/2018-09-14-Computable-posts-from-quansight.ipynb
        post = Path('../_posts/2018-09-14-Computable-posts-from-quansight.md')
        assert post.exists()
        *_, fm, md = post.read_text().split('---', 2)
        fm = yaml.safe_load(io.StringIO(fm))
        assert isinstance(fm, dict)
        assert 'layout' in fm, "The blog post won't show with Jekyll."

    if __name__ == '__main__':
        !source activate p6 && ipython -m pytest -- 2018-09-14-Computable-posts-from-quansight.ipynb